In [1]:
import numpy as np
import pandas as pd
import re
import nltk
import keras
from keras.applications.resnet50 import ResNet50, preprocess_input, decode_predictions
from keras.preprocessing import image
from keras.models import Sequential, Model
from keras.layers import Dense
from time import time
import pickle
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

Using TensorFlow backend.
/home/avinash/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/avinash/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/avinash/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/avinash/.local/lib/python3.6/site-packages/tensorflow/py

In [2]:
with open("Flickr_TextData/Flickr8k.token.txt") as f:
    captions = f.read()

In [3]:
captions = captions.split("\n")[:-1]

In [4]:
len(captions)
captions[0]

'1000268201_693b08cb0e.jpg#0\tA child in a pink dress is climbing up a set of stairs in an entry way .'

In [5]:
descriptions = {}

for element in captions:
    i_to_c = element.split("\t")
    cap = i_to_c[1]
    
    key = i_to_c[0].split(".")[0]
    
    if descriptions.get(key) == None:
        descriptions[key] = []
        
    descriptions[key].append(cap)

In [6]:
descriptions["1000268201_693b08cb0e"]

['A child in a pink dress is climbing up a set of stairs in an entry way .',
 'A girl going into a wooden building .',
 'A little girl climbing into a wooden playhouse .',
 'A little girl climbing the stairs to her playhouse .',
 'A little girl in a pink dress going into a wooden cabin .']

## Data Cleaning

In [7]:
"""
1. lower
2. remove puntuations
3. remove words less than length 2
"""

def clean_text(sample):
    sample = sample.lower()
    sample = re.sub("[^a-z]+"," ", sample)
    sample = sample.split()
    
    sample = [s for s in sample if len(s)>1]
    
    sample = " ".join(sample)
    
    return sample


In [8]:
for key, desc_list in descriptions.items():
    for i in range(len(desc_list)):
        descriptions[key][i] = clean_text(desc_list[i])

In [9]:
descriptions["1000268201_693b08cb0e"]

['child in pink dress is climbing up set of stairs in an entry way',
 'girl going into wooden building',
 'little girl climbing into wooden playhouse',
 'little girl climbing the stairs to her playhouse',
 'little girl in pink dress going into wooden cabin']

In [10]:
# create unique vocabulary

vocabulary = set()

for key in descriptions.keys():
    [vocabulary.update(i.split()) for i in descriptions[key]]
    
print(len(vocabulary))

8424


In [11]:
all_vocab = []

for key in descriptions.keys():
    [all_vocab.append(i) for des in descriptions[key] for i in des.split()]
    
print(len(all_vocab))

373837


In [12]:
import collections

In [13]:
counter = collections.Counter(all_vocab)

In [14]:
dict_ = dict(counter)

In [15]:
sorted_dic = sorted(dict_.items(),reverse=True,key = lambda x : x[1])
threshhold_val = 10
sorted_dic = [x for x in sorted_dic if x[1]>threshhold_val]
vocabulary = [x[0] for x in sorted_dic]

In [16]:
len(vocabulary)

1845

# Load Training Testing Data

In [17]:
with open("Flickr_TextData/Flickr_8k.trainImages.txt") as f:
    train = f.read()
with open("Flickr_TextData/Flickr_8k.testImages.txt") as f:
    test = f.read()

In [18]:
train = [e.split(".")[0] for e in train.split("\n")[:-1]]
test = [e.split(".")[0] for e in test.split("\n")[:-1]]

In [19]:
train_descriptions = {}
for t in train:
    train_descriptions[t] = []
    
    for cap in descriptions[t]:
        cap_append = "startseq " + cap + " endseq"
        train_descriptions[t].append(cap_append)

In [20]:
train_descriptions["1000268201_693b08cb0e"]

['startseq child in pink dress is climbing up set of stairs in an entry way endseq',
 'startseq girl going into wooden building endseq',
 'startseq little girl climbing into wooden playhouse endseq',
 'startseq little girl climbing the stairs to her playhouse endseq',
 'startseq little girl in pink dress going into wooden cabin endseq']

## Data Preprocessing

In [21]:
model = ResNet50(weights = "imagenet",input_shape=(224,224,3))

In [22]:
model_new = Model(inputs = model.input,outputs=model.layers[-2].output)

In [23]:
images = "flickr_data/Flickr_Data/Images/"

In [24]:
def preprocess_image(img):
    img = image.load_img(img, target_size=(224,224))
    img = image.img_to_array(img)
    img = np.expand_dims(img,axis=0)
    img = preprocess_input(img)
    
    return img

In [25]:
def encode_image(img):
    img = preprocess_image(img)
    feature_vector = model_new.predict(img).reshape((2048,))
    return feature_vector

In [26]:
x = encode_image(images+"/1000268201_693b08cb0e.jpg")

In [27]:
x.shape

(2048,)

In [28]:
start = time()

encoding_train = {}

for ix,img_ in enumerate(train):
    
    img = images+img_+".jpg"
    
    encoding_train[img] = encode_image(img)
    
    if ix%100 ==0:
        print("encoding image "+str(ix),end="\r")
    


In [29]:
# writing pkl file on disk
with open("encoded_train_images.pkl", "wb") as p:
    pickle.dump(encoding_train,p)

In [30]:
#reading pkl file from disk
with open("encoded_train_images.pkl", "rb") as p:
    encoding_train = pickle.load(p)

In [31]:
start = time()

encoding_test = {}

for ix,img_ in enumerate(test):
    
    img = images+img_+".jpg"
    
    encoding_test[img] = encode_image(img)
    
    if ix%100 ==0:
        print("encoding image "+str(ix),end="\r")
    


In [32]:
# writing pkl file on disk
with open("encoded_test_images.pkl", "wb") as p:
    pickle.dump(encoding_test,p)

In [33]:
#reading pkl file from disk
with open("encoded_test_images.pkl", "rb") as p:
    encoding_test = pickle.load(p)

# Data Preprocessing Captions

In [34]:
# Mapping word_to_idx and idx_to_word

ix = 1
word_to_idx = {}
idx_to_word = {}

for e in vocabulary:
    word_to_idx[e] = ix
    idx_to_word[ix] = e
    ix+=1

In [35]:
word_to_idx['dog']

6

In [36]:
idx_to_word[1846] = "startseq"
word_to_idx["startseq"] = 1846

idx_to_word[1847] = "endseq"
word_to_idx["endseq"] = 1847

In [37]:
vocab_size = len(idx_to_word) +1
print(vocab_size)

1848


In [38]:
all_caption_len = []

for key in train_descriptions.keys():
    for cap in train_descriptions[key]:
        all_caption_len.append(len(cap.split()))
        
max_len = max(all_caption_len)
print(max_len)

35


## Data Prepare using Generator

In [39]:
def data_generator(train_descriptions,encoding_train,word_to_idx, max_len,num_photos_per_batch):
    x1,x2,y = [],[],[]
    n=0
    while True:
        for key, desc_list in train_descriptions.items():
            n+=1
            photo = encoding_train[key+".jpg"]
            for desc in desc_list:
                seq = [word_to_idx[word] for word in desc.split() if word in word_to_idx]
                for i in range(1,len(seq)):
                    in_seq = seq[0:i]
                    out_seq = seq[i]
                    in_seq = pad_sequences([in_seq],maxlen=max_len,value=0,padding="post")[0]
                    out_seq = to_categorical([out_seq],num_classes=vocab_size)[0]
                    x1.append(photo)
                    x2.append(in_seq)
                    y.append(out_seq)
            if n==num_photos_per_batch:
                yield [[np.array(x1),np.array(x2)],np.array(y)]
                x1,x2,y =[],[],[]
                n=0
                

# Word Embedding

In [40]:
with open("glove.6B.50d.txt",encoding="utf8") as f:
    embedding_index={}
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:],dtype="float")
        embedding_index[word] = coefs